In [124]:
#start with a few basic imports (pandas, sci-kit learn)
from bs4 import BeautifulSoup as BS
import urllib.request
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression


In [125]:
#populate the datapoints of a particular team 
def getTeamData(url):
    html = requests.get(url).text
    soup = BS(html, 'html.parser')
    placeholders = soup.find_all('div', {'class': 'placeholder'})
    total_tables = 0
    for x in placeholders:
        comment = ''.join(x.next_siblings)
        soup_comment = BS(comment, 'html.parser')
        table = soup_comment.find('table')
        trueTable = soup_comment.find('table', id="team_misc")
        if (trueTable != None):
            break

    teamList = [];

    for tr in trueTable.findAll('tr'):
        for td in tr.findAll('td'):
            teamList.append(td.text)

    teamList = teamList[:20]
    teamSeries = pd.Series(teamList);
    return teamSeries

In [126]:
#list of teams in the league 
teamList = ["TOR","BOS","NYK","BRK","CLE","IND","PHI","DET","CHI","MIL","MIA","ATL","CHO","WAS","ORL","OKC","POR","UTA","DEN","MIN","LAC","SAC","PHO","LAL","SAS","DAL","MEM","HOU","NOP","GSW"]

In [127]:
#populates a series of 30 teams with all datapoints 
fullTeamsSeries = pd.Series();
for teamID in teamList:
    currentURL = "https://www.basketball-reference.com/teams/" + teamID + "/2018.html"
    fullTeamsSeries.loc[teamID] = getTeamData(currentURL)

print(fullTeamsSeries)

    
    
    
    

TOR    0        59
1        23
2        60
3        22
4      7.78
5     -0.49
6      7.29
7     113.8
8     105.9
9      97.4
10     .250
11     .377
12     .539
13     12.1
14     23.0
15     .198
16     .501
17     13.0
18     77.7
19     .212
dtype: object
BOS    0        55
1        27
2        51
3        31
4      3.59
5     -0.35
6      3.23
7     107.6
8     103.9
9      96.0
10     .243
11     .357
12     .518
13     13.0
14     21.5
15     .188
16     .495
17     13.0
18     78.4
19     .191
dtype: object
NYK    0        29
1        53
2        32
3        50
4     -3.56
5      0.03
6     -3.53
7     107.1
8     110.7
9      96.8
10     .216
11     .266
12     .510
13     13.3
14     24.1
15     .170
16     .522
17     12.2
18     76.7
19     .208
dtype: object
BRK    0        28
1        54
2        31
3        51
4     -3.74
5      0.07
6     -3.67
7     106.9
8     110.6
9      98.9
10     .260
11     .411
12     .514
13     13.6
14     21.0
15     .201
16     .517
17    

In [128]:
def soupTableToDF(table_soup, header):
    """
    Parses the HTML/Soup table for the gamelog stats.
    Returns a pandas DataFrame
    """
    if not table_soup:
        return None
    else:
        rows = table_soup[0].findAll('tr')[1:]  # all rows but the header

        # remove blank rows
        rows = [r for r in rows if len(r.findAll('td')) > 0]

        # build 2d list of table values
        parsed_rows = [[col.getText() for col in row.findAll('td')] for row in rows]
        parsed_table = [row for row in parsed_rows if row[0] != ""]
        try:
            return pd.DataFrame.from_records(parsed_table, columns=header).dropna(subset=["G"])
        except Exception as e:
            print("ERROR - Couldn't create dataframe:", e)
            print(parsed_table)
            return None



In [129]:
#have to read in a dataframe of teams playing and the results of the game - team 1 = 0 and team 2 = 1 -> 
#if team 1 wins output 0 and if te']
dfs = pd.DataFrame();
seasonMonths = ['october','november','december','january','february','march','april','may','june']
for month in seasonMonths:
    monthURL = 'https://www.basketball-reference.com/leagues/NBA_2018_games-' + month + '.html';
    currentmonthDFS = pd.read_html(monthURL)[0]
    currentmonthDFS = currentmonthDFS[['Visitor/Neutral','PTS','Home/Neutral','PTS.1']]
    dfs = pd.concat([dfs, currentmonthDFS], ignore_index=True)





    
    

In [130]:
teamDict = {"Atlanta Hawks":"ATL", 
"Boston Celtics":"BOS", 
"Brooklyn Nets" : "BRK",
"Charlotte Hornets": "CHO", 
"Chicago Bulls":"CHI", 
"Cleveland Cavaliers":"CLE", 
"Dallas Mavericks":"DAL", 
"Denver Nuggets":"DEN", 
"Detroit Pistons":"DET", 
"Golden State Warriors":"GSW",
"Houston Rockets":"HOU", 
"Indiana Pacers":"IND", 
"Los Angeles Clippers":"LAC", 
"Los Angeles Lakers":"LAL", 
"Memphis Grizzlies":"MEM", 
"Miami Heat":"MIA", 
"Milwaukee Bucks":"MIL", 
"Minnesota Timberwolves":"MIN", 
"New Orleans Pelicans":"NOP", 
"New York Knicks":"NYK",
"Oklahoma City Thunder":"OKC", 
"Orlando Magic":"ORL", 
"Philadelphia 76ers":"PHI", 
"Phoenix Suns":"PHO", 
"Portland Trail Blazers":"POR", 
"Sacramento Kings":"SAC", 
"San Antonio Spurs":"SAS", 
"Toronto Raptors":"TOR",
"Utah Jazz":"UTA", 
"Washington Wizards":"WAS"}

In [131]:
def addColumn(row):
    if row['PTS'] > row['PTS.1']:
        return 0;
    if (row['PTS.1'] > row['PTS']):
        return 1;

In [132]:
dfs['win/loss label'] = dfs.apply (lambda row: addColumn (row),axis=1)
dfs = dfs.drop('PTS',axis=1)
dfs = dfs.drop('PTS.1',axis=1)


In [133]:
col_names = ['X-Comp','Y-Comp']
teamDataFrame = pd.DataFrame(columns=col_names)
for index, row in dfs.iterrows():
    if (index == 1230):
        continue
    teamVariable1 = row["Visitor/Neutral"]
    teamVariable2 = row["Home/Neutral"]
    teamVariable3 = row["win/loss label"]
    teamShort1 = teamDict.get(teamVariable1)
    teamShort2 = teamDict.get(teamVariable2)
    team1Array = []
    team2Array = []
    for number in fullTeamsSeries[teamShort1]:
        team1Array.append(float(number))
    for number in fullTeamsSeries[teamShort2]:
        team2Array.append(float(number))
    finalList = [x - y for x, y in zip(team1Array, team2Array)]
    teamDictionary = {'X-Comp': finalList, 'Y-Comp': float(teamVariable3)}
    teamDataFrame.loc[len(teamDataFrame)] = teamDictionary
    
print(teamDataFrame)

                                                                                                                                                                                                                                                                                                                                                                                  X-Comp  \
0                                                                                         [5.0, -5.0, 8.0, -8.0, 2.65, 0.0, 2.64, -5.300000000000011, -8.0, -2.0, -0.03200000000000003, -0.02200000000000002, -0.029000000000000026, 0.40000000000000036, 1.3999999999999986, -0.025999999999999995, -0.04500000000000004, 0.8000000000000007, 1.1000000000000085, 0.024999999999999994]   
1                                                                            [7.0, -7.0, 5.0, -5.0, 2.5, -0.08000000000000002, 2.420000000000001, 1.1000000000000085, -1.5, -2.0, 0.059, 0.16299999999999998, -0.017999999999999905, -1.40000000

In [139]:
y = teamDataFrame['Y-Comp']
X = teamDataFrame['X-Comp']